### Начало обучение NLP  с помощью пакета spaCY
###### часть 1 Глава 2. Конвейер обработки текста
Старт: 01.10.2022

In [28]:
import spacy as sp

In [29]:
nlp = sp.load('ru_core_news_lg')

# Конвеер
### Токенезация


In [30]:
# демонстрация токенезации

doc = nlp('Привет, меня зовут Денис')
print([w.text for w in doc])

['Привет', ',', 'меня', 'зовут', 'Денис']


### Лемматизация

In [31]:
# пример лемматизации

doc = nlp(u'Я вижу красивейший закат и балдею и влюбляюсь в Питер')
for token in doc:
    print(token.text, token.lemma_)


Я я
вижу видеть
красивейший красивый
закат закат
и и
балдею балдеть
и и
влюбляюсь влюбляться
в в
Питер питер


Пимер: бот для бронирования авиобилетов. Нужно счиатать строку и добавить в исключение названия городов. Допустим слово
"Питер" должно при леммизации сделаться "Санкт-Петербург"

In [32]:
input_str = u'Я хочу купить билет в Питер'
doc = nlp(input_str )

print([w.text for w in doc]) # ['Я', 'хочу', 'купить', 'билет', 'в', 'Питер'] - плохо

nlp.get_pipe("attribute_ruler").add([[{"TEXT": "Питер"}]], {"LEMMA": "Санкт-Петербург"})

doc_ru = nlp(input_str)    
print(['token:%s lemma:%s' % (t.text, t.lemma_) for t in doc])


print([w.lemma_ for w in nlp(input_str)])

['Я', 'хочу', 'купить', 'билет', 'в', 'Питер']
['token:Я lemma:я', 'token:хочу lemma:хотеть', 'token:купить lemma:купить', 'token:билет lemma:билет', 'token:в lemma:в', 'token:Питер lemma:питер']
['я', 'хотеть', 'купить', 'билет', 'в', 'Санкт-Петербург']


### Частеречная разметка

In [42]:
# частеречная разметка
nlp_en = sp.load('en_core_web_sm')
doc = nlp_en(u'I have flown to LA. Now I am flying to Frisco')

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_)

print([w.text for w in doc if w.tag_=='VBG' or w.pos_ =='VB'])

I I PRON PRP
have have AUX VBP
flown fly VERB VBN
to to ADP IN
LA LA PROPN NNP
. . PUNCT .
Now now ADV RB
I I PRON PRP
am be AUX VBP
flying fly VERB VBG
to to ADP IN
Frisco Frisco PROPN NNP
['flying']


Чтобы посмотреть какие отноешния spacy установил между словами при частиречной разметке. Посмотрим метки синтаксичечкой зависимости.

ROOT - смысловой глагол в предложении 
nsubj - подлежащие 

Эти два тега есть  в люьом законченном предложении, однако не все предложения на пракктике законченные.

In [41]:
print('input str: ')
print(doc)
print(5*'----')
for token in doc:
    print(token.head.text, token.dep_, token.text)

input str: 
I have flown to LA. Now I am flying to Frisco.
--------------------
I ROOT I
I flat:foreign have
I flat:foreign flown
I flat:foreign to
I flat:foreign LA
I punct .
Now ROOT Now
Now flat:foreign I
Now flat:foreign am
Now flat:foreign flying
Now flat:foreign to
Now flat:foreign Frisco
Now punct .


In [35]:
print('input str: ')
print(doc_ru)
print(5*'----')
for token in doc_ru:
    print(token.head.text, token.dep_, token.text)

input str: 
Я хочу купить билет в Питер
--------------------
хочу nsubj Я
хочу ROOT хочу
хочу xcomp купить
купить obj билет
Питер case в
купить obl Питер


Заметим, что главная мысль предложения в метках root и obj(nobj in en);

Попробуем находить слова, соответствующие этим зависимостям.

In [54]:
doc_en = nlp_en(u'I have flown to LA. Now I am flying to Frisco.')

for token in doc_en:
    print(token.head.text, token.dep_, token.text)

for sent in doc_en.sents:
    print('--------новое предложение------------')
    print([w.text for w in sent if w.dep_ == 'ROOT' or w.dep_ == 'pobj'])

flown nsubj I
flown aux have
flown ROOT flown
flown prep to
to pobj LA
flown punct .
flying advmod Now
flying nsubj I
flying aux am
flying ROOT flying
flying prep to
to pobj Frisco
flying punct .
--------новое предложение------------
['flown', 'LA']
--------новое предложение------------
['flying', 'Frisco']


In [58]:
doc = nlp(u'Меня зовут Денис. Я хочу полететь в Питер, потому что я устал и кот болеет. Кот улетел в Питер.')

for token in doc:
    print(token.head.text, token.dep_, token.text)

for sent in doc.sents: #doc.sents разбивает текст по предложениям
    print('--------новое предложение------------')
    # печатаем главный смысл
    print([w.text for w in sent if w.dep_ == 'ROOT' or w.dep_ == 'obj' or  w.dep_ == 'obl' or  w.dep_ == 'xcomp'])

зовут obj Меня
зовут ROOT зовут
зовут nsubj Денис
зовут punct .
хочу nsubj Я
хочу ROOT хочу
хочу xcomp полететь
Питер case в
полететь obl Питер
устал punct ,
устал mark потому
потому fixed что
устал nsubj я
полететь advcl устал
болеет cc и
болеет nsubj кот
устал conj болеет
хочу punct .
улетел nsubj Кот
улетел ROOT улетел
Питер case в
улетел obl Питер
улетел punct .
--------новое предложение------------
['Меня', 'зовут']
--------новое предложение------------
['хочу', 'полететь', 'Питер']
--------новое предложение------------
['улетел', 'Питер']


#### Распознование именовоных сущностей

**Именованная сущность** (named entity) — объект, на который можно ссылаться по его собственному наименованию. Именованной сущностью может быть человек, организация, место или другая сущность. Именованные сущности играют важную роль в NLP, поскольку позволяют выяснить, о каком месте или организации говорит пользователь.

In [64]:
doc = nlp_en(u'I have flown to LA. Now I am flying to Frisco.')
for token in doc:
    if token.ent_type != 0:
        print(token.text, token.ent_type_)
print('GPE - Геополитическая сущность')

LA GPE
Frisco ORG
GPE - Геополитическая сущность


Если атрибуту ent_type токена не присвоено значение 0, значит, этот токен — именованная сущность. В таком случае выводим атрибут ent_type_

In [63]:
doc = nlp(u'Меня зовут Денис. Я хочу полететь в Питер, потому что я устал и кот болеет. Кот улетел в Питер.')
for token in doc:
    if token.ent_type != 0:
        print(token.text, token.ent_type_)

# PER - персона
# LOC - локация

Денис PER
Питер LOC
Питер LOC
